- EV Sales data url:
    - [https://api.iea.org/evs?parameters=EV-sales&category=Historical&mode=Cars&csv=true](https://api.iea.org/evs?parameters=EV%20sales&category=Historical&mode=Cars&csv=true)
- EV Charging Points url:
    - [https://api.iea.org/evs?parameters=EV charging points&category=Historical&mode=EV&csv=true](https://api.iea.org/evs?parameters=EV%20charging%20points&category=Historical&mode=EV&csv=true)

### Develop load_data() function

In [314]:
ev_url = "https://api.iea.org/evs?parameters=EV-sales&category=Historical&mode=Cars&csv=true"

ev_charging_points_url = "https://api.iea.org/evs?parameters=EV-charging-points&category=Historical&mode=EV&csv=true"


In [315]:
import pandas as pd

In [316]:
ev_df = pd.read_csv(ev_url)

ev_df.head()

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000


In [317]:
# print the classes from the categorical columns
ev_df.dtypes

region         object
category       object
parameter      object
mode           object
powertrain     object
year            int64
unit           object
value         float64
dtype: object

In [318]:
cat_cols = ev_df.select_dtypes(include=['object']).columns

numeric_cols = ev_df.select_dtypes(include=['number']).columns

In [319]:
# loop through the categorical columns and print the unique values

for col in cat_cols:
    print(f"Column: {col}")
    print(f"{col}: {ev_df[col].unique()}")
    print("\n")

Column: region
region: ['Australia' 'Austria' 'Belgium' 'Brazil' 'Bulgaria' 'Canada' 'Chile'
 'China' 'Colombia' 'Costa Rica' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece'
 'Hungary' 'Iceland' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Korea'
 'Latvia' 'Lithuania' 'Luxembourg' 'Mexico' 'Netherlands' 'New Zealand'
 'Norway' 'Poland' 'Portugal' 'Rest of the world' 'Romania' 'Seychelles'
 'Slovakia' 'Slovenia' 'South Africa' 'Spain' 'Sweden' 'Switzerland'
 'Turkiye' 'United Arab Emirates' 'United Kingdom' 'USA' 'World']


Column: category
category: ['Historical']


Column: parameter
parameter: ['EV sales' 'EV stock share' 'EV sales share' 'EV stock'
 'Electricity demand' 'Oil displacement Mbd'
 'Oil displacement, million lge']


Column: mode
mode: ['Cars']


Column: powertrain
powertrain: ['BEV' 'EV' 'PHEV' 'FCEV']


Column: unit
unit: ['Vehicles' 'percent' 'GWh' 'Milion barrels per day'
 'Oil displacement, million lge']




In [320]:
# loop through the numeric columns and print the description
for col in numeric_cols:
    print(f"Column: {col}")
    print(f"{col}: {ev_df[col].describe()}")
    print("\n")
    

Column: year
year: count    3798.000000
mean     2017.369932
std         3.809226
min      2010.000000
25%      2014.000000
50%      2018.000000
75%      2021.000000
max      2023.000000
Name: year, dtype: float64


Column: value
value: count    3.798000e+03
mean     1.009542e+05
std      8.184402e+05
min      1.500000e-05
25%      2.300000e+00
50%      1.900000e+02
75%      6.800000e+03
max      2.800000e+07
Name: value, dtype: float64




In [321]:
ev_df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,4.900000e+01
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,3.900000e-04
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,6.500000e-03
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,4.900000e+01
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,2.200000e+02
...,...,...,...,...,...,...,...,...
3793,World,Historical,EV stock,Cars,BEV,2023,Vehicles,2.800000e+07
3794,World,Historical,EV sales,Cars,BEV,2023,Vehicles,9.500000e+06
3795,World,Historical,EV sales share,Cars,EV,2023,percent,1.800000e+01
3796,World,Historical,EV stock share,Cars,EV,2023,percent,3.200000e+00


In [322]:
# select the columns we want to keep
ev_df = ev_df[['region', 'parameter', 'powertrain', 'year', 'unit', 'value']]

# split the dataframe into two based on the parameter
ev_sales_df = ev_df[ev_df['parameter'] == 'EV sales']
ev_sales_share_df = ev_df[ev_df['parameter'] == 'EV sales share']

# drop the parameter and unit columns
ev_sales_share_df = ev_sales_share_df.drop(columns=['parameter', 'unit'])
ev_sales_df = ev_sales_df.drop(columns=['parameter', "unit"])

In [323]:
# rename the value column to sales
ev_sales_df.rename(columns={'value': 'sales'}, inplace=True)
ev_sales_share_df.rename(columns={'value': 'sales_share'}, inplace=True)

display(ev_sales_df)
display(ev_sales_share_df)

,region,powertrain,year,sales
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0
...,...,...,...,...
3781,World,FCEV,2022,15000.0
3783,World,PHEV,2022,2900000.0
3788,World,PHEV,2023,4300000.0
3792,World,FCEV,2023,8900.0


,region,powertrain,year,sales_share
2,Australia,EV,2011,0.0065
7,Australia,EV,2012,0.0300
12,Australia,EV,2013,0.0340
19,Australia,EV,2014,0.1600
24,Australia,EV,2015,0.2000
...,...,...,...,...
3751,World,EV,2019,2.6000
3756,World,EV,2020,4.2000
3773,World,EV,2021,8.9000
3778,World,EV,2022,14.0000


In [324]:
def load_data():
    ev_url = "https://api.iea.org/evs?parameters=EV-sales&category=Historical&mode=Cars&csv=true"

    ev_df = pd.read_csv(ev_url)

    ev_df = ev_df[['region', 'parameter', 'powertrain', 'year', 'unit', 'value']]

    # split the dataframe into two based on the parameter
    ev_sales_df = ev_df[ev_df['parameter'] == 'EV sales']
    ev_sales_share_df = ev_df[ev_df['parameter'] == 'EV sales share']

    # drop the parameter and unit columns
    ev_sales_share_df = ev_sales_share_df.drop(columns=['parameter', 'unit'])
    ev_sales_df = ev_sales_df.drop(columns=['parameter', "unit"])

    # rename the value column to sales
    ev_sales_df.rename(columns={'value': 'sales'}, inplace=True)
    ev_sales_share_df.rename(columns={'value': 'sales_share'}, inplace=True)

    # reset the index
    ev_sales_df.reset_index(drop=True, inplace=True)
    ev_sales_share_df.reset_index(drop=True, inplace=True)

    return ev_sales_df, ev_sales_share_df

In [325]:
ev_sales_df, ev_sales_share_df = load_data()

In [326]:
ev_sales_df

,region,powertrain,year,sales
0,Australia,BEV,2011,49.0
1,Australia,PHEV,2012,80.0
2,Australia,BEV,2012,170.0
3,Australia,BEV,2013,190.0
4,Australia,PHEV,2013,100.0
...,...,...,...,...
1337,World,FCEV,2022,15000.0
1338,World,PHEV,2022,2900000.0
1339,World,PHEV,2023,4300000.0
1340,World,FCEV,2023,8900.0


In [327]:
ev_sales_share_df

,region,powertrain,year,sales_share
0,Australia,EV,2011,0.0065
1,Australia,EV,2012,0.0300
2,Australia,EV,2013,0.0340
3,Australia,EV,2014,0.1600
4,Australia,EV,2015,0.2000
...,...,...,...,...
598,World,EV,2019,2.6000
599,World,EV,2020,4.2000
600,World,EV,2021,8.9000
601,World,EV,2022,14.0000


### Develop sales_by_region() function

In [328]:
# group by year and country and count the "Vehicle" values

ev_df

# From ev_df, filter out EU27, Europe, Rest of the world and World
exclude_list = ['Europe', 'EU27', 'Rest of the world', 'World']
ev_sales_by_region_df = ev_df[~ev_df['region'].isin(exclude_list)]

ev_sales_by_region_df

,region,parameter,powertrain,year,unit,value
0,Australia,EV sales,BEV,2011,Vehicles,4.900000e+01
1,Australia,EV stock share,EV,2011,percent,3.900000e-04
2,Australia,EV sales share,EV,2011,percent,6.500000e-03
3,Australia,EV stock,BEV,2011,Vehicles,4.900000e+01
4,Australia,EV stock,BEV,2012,Vehicles,2.200000e+02
...,...,...,...,...,...,...
3639,USA,Oil displacement Mbd,EV,2023,Milion barrels per day,1.700000e-01
3640,USA,"Oil displacement, million lge",EV,2023,"Oil displacement, million lge",9.700000e+03
3641,USA,EV sales,PHEV,2023,Vehicles,2.900000e+05
3642,USA,EV stock,PHEV,2023,Vehicles,1.300000e+06


In [329]:
# in ev_sales_by_region_df show me only rows where parameter is "EV sales"
ev_sales_by_region_df = ev_sales_by_region_df[ev_sales_by_region_df['parameter'] == "EV sales"]
ev_sales_by_region_df

,region,parameter,powertrain,year,unit,value
0,Australia,EV sales,BEV,2011,Vehicles,49.0
6,Australia,EV sales,PHEV,2012,Vehicles,80.0
9,Australia,EV sales,BEV,2012,Vehicles,170.0
10,Australia,EV sales,BEV,2013,Vehicles,190.0
13,Australia,EV sales,PHEV,2013,Vehicles,100.0
...,...,...,...,...,...,...
3629,USA,EV sales,FCEV,2022,Vehicles,2700.0
3630,USA,EV sales,BEV,2022,Vehicles,800000.0
3635,USA,EV sales,BEV,2023,Vehicles,1100000.0
3636,USA,EV sales,FCEV,2023,Vehicles,3000.0


In [330]:
ev_sales_by_region_df['unit'].unique()

array(['Vehicles'], dtype=object)

In [331]:
# in ev_sales_by_region_df remove colum "parameter"
ev_sales_by_region_df = ev_sales_by_region_df.drop(columns=['parameter', 'unit'])
ev_sales_by_region_df

,region,powertrain,year,value
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0
...,...,...,...,...
3629,USA,FCEV,2022,2700.0
3630,USA,BEV,2022,800000.0
3635,USA,BEV,2023,1100000.0
3636,USA,FCEV,2023,3000.0


In [332]:
# change name of column "value" to "sales"
ev_sales_by_region_df.rename(columns={'value': 'sales'}, inplace=True)
ev_sales_by_region_df


,region,powertrain,year,sales
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0
...,...,...,...,...
3629,USA,FCEV,2022,2700.0
3630,USA,BEV,2022,800000.0
3635,USA,BEV,2023,1100000.0
3636,USA,FCEV,2023,3000.0


In [333]:
ev_sales_by_region_df["region"].unique()

array(['Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada',
       'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'India', 'Ireland',
       'Israel', 'Italy', 'Japan', 'Korea', 'Latvia', 'Lithuania',
       'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand', 'Norway',
       'Poland', 'Portugal', 'Romania', 'Seychelles', 'Slovakia',
       'Slovenia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
       'Turkiye', 'United Arab Emirates', 'United Kingdom', 'USA'],
      dtype=object)

In [334]:
def sales_by_region():
    ev_url = "https://api.iea.org/evs?parameters=EV-sales&category=Historical&mode=Cars&csv=true"

    ev_df = pd.read_csv(ev_url)

    ev_df = ev_df[['region', 'parameter', 'powertrain', 'year', 'unit', 'value']]

    exclude_list = ['Europe', 'EU27', 'Rest of the world', 'World']
    ev_sales_by_region_df = ev_df[~ev_df['region'].isin(exclude_list)]

    ev_sales_by_region_df = ev_sales_by_region_df[ev_sales_by_region_df['parameter'] == "EV sales"]

    ev_sales_by_region_df.rename(columns={'value': 'sales'}, inplace=True)

    ev_sales_by_region_df = ev_sales_by_region_df.drop(columns=['parameter', 'unit'])

    return ev_sales_by_region_df


In [335]:
sales_by_region()

,region,powertrain,year,sales
0,Australia,BEV,2011,49.0
6,Australia,PHEV,2012,80.0
9,Australia,BEV,2012,170.0
10,Australia,BEV,2013,190.0
13,Australia,PHEV,2013,100.0
...,...,...,...,...
3629,USA,FCEV,2022,2700.0
3630,USA,BEV,2022,800000.0
3635,USA,BEV,2023,1100000.0
3636,USA,FCEV,2023,3000.0


In [336]:
ev_sales_by_region_df.groupby("region")["sales"].sum().nlargest(10)

region
China             21775929.0
USA                4763255.0
Germany            2913809.0
United Kingdom     1584498.0
France             1542434.0
Norway              849645.0
Sweden              682392.0
Netherlands         652237.2
Japan               611749.0
Canada              564997.0
Name: sales, dtype: float64

### Loadinf charging points data

In [337]:
ev_charging_points_url = "https://api.iea.org/evs?parameters=EV-charging-points&category=Historical&mode=EV&csv=true"

ev_charging_points_df = pd.read_csv(ev_charging_points_url)

ev_charging_points_df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available slow,2019,charging points,1700.0
...,...,...,...,...,...,...,...,...
745,World,Historical,EV charging points,EV,Publicly available fast,2021,charging points,560000.0
746,World,Historical,EV charging points,EV,Publicly available fast,2022,charging points,900000.0
747,World,Historical,EV charging points,EV,Publicly available slow,2022,charging points,1800000.0
748,World,Historical,EV charging points,EV,Publicly available slow,2023,charging points,2500000.0


In [338]:
# loop through the categorical columns in ev_charging_points_df and print the unique values 
for col in ev_charging_points_df.select_dtypes(include=['object']).columns:
    print(f"Column: {col}")
    print(f"{col}: {ev_charging_points_df[col].unique()}")
    print("\n")
# loop through the numeric columns in ev_charging_points_df and print the description
for col in ev_charging_points_df.select_dtypes(include=['number']).columns:
    print(f"Column: {col}")
    print(f"{col}: {ev_charging_points_df[col].describe()}")

Column: region
region: ['Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'
 'Denmark' 'EU27' 'Europe' 'Finland' 'France' 'Germany' 'Greece' 'Iceland'
 'India' 'Indonesia' 'Israel' 'Italy' 'Japan' 'Korea' 'Mexico'
 'Netherlands' 'New Zealand' 'Norway' 'Poland' 'Portugal' 'South Africa'
 'Spain' 'Sweden' 'Switzerland' 'Thailand' 'Turkiye' 'United Kingdom'
 'USA' 'World']


Column: category
category: ['Historical']


Column: parameter
parameter: ['EV charging points']


Column: mode
mode: ['EV']


Column: powertrain
powertrain: ['Publicly available fast' 'Publicly available slow']


Column: unit
unit: ['charging points']


Column: year
year: count     750.000000
mean     2017.924000
std         3.483064
min      2010.000000
25%      2015.000000
50%      2018.000000
75%      2021.000000
max      2023.000000
Name: year, dtype: float64
Column: value
value: count    7.500000e+02
mean     3.827495e+04
std      1.741407e+05
min      1.000000e-01
25%      1.400000e+02
50%      1.4

In [339]:
ev_charging_points_df = ev_charging_points_df[['region', 'powertrain', 'year', 'value']]
ev_charging_points_df

,region,powertrain,year,value
0,Australia,Publicly available fast,2017,40.0
1,Australia,Publicly available slow,2017,440.0
2,Australia,Publicly available fast,2018,61.0
3,Australia,Publicly available slow,2018,670.0
4,Australia,Publicly available slow,2019,1700.0
...,...,...,...,...
745,World,Publicly available fast,2021,560000.0
746,World,Publicly available fast,2022,900000.0
747,World,Publicly available slow,2022,1800000.0
748,World,Publicly available slow,2023,2500000.0


In [340]:
# change name of column "value" to "charging_points"
ev_charging_points_df.rename(columns={'value': 'charging_points'}, inplace=True)
ev_charging_points_df

/var/folders/yz/shh48k7s5kg3qtp0x9hzjb6c0000gn/T/ipykernel_75727/4021729360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_charging_points_df.rename(columns={'value': 'charging_points'}, inplace=True)


,region,powertrain,year,charging_points
0,Australia,Publicly available fast,2017,40.0
1,Australia,Publicly available slow,2017,440.0
2,Australia,Publicly available fast,2018,61.0
3,Australia,Publicly available slow,2018,670.0
4,Australia,Publicly available slow,2019,1700.0
...,...,...,...,...
745,World,Publicly available fast,2021,560000.0
746,World,Publicly available fast,2022,900000.0
747,World,Publicly available slow,2022,1800000.0
748,World,Publicly available slow,2023,2500000.0


In [341]:
exclude_list = ['Europe', 'EU27', 'Rest of the world', 'World']
ev_charging_points_df = ev_charging_points_df[~ev_charging_points_df['region'].isin(exclude_list)]

ev_charging_points_df

,region,powertrain,year,charging_points
0,Australia,Publicly available fast,2017,40.0
1,Australia,Publicly available slow,2017,440.0
2,Australia,Publicly available fast,2018,61.0
3,Australia,Publicly available slow,2018,670.0
4,Australia,Publicly available slow,2019,1700.0
...,...,...,...,...
717,USA,Publicly available slow,2021,92000.0
718,USA,Publicly available fast,2022,28000.0
719,USA,Publicly available slow,2022,100000.0
720,USA,Publicly available fast,2023,43000.0


In [342]:
def load_charging_points_data():
    ev_charging_points_url = "https://api.iea.org/evs?parameters=EV-charging-points&category=Historical&mode=EV&csv=true"

    ev_charging_points_df = pd.read_csv(ev_charging_points_url)

    ev_charging_points_df = ev_charging_points_df[['region', 'powertrain', 'year', 'value']]
    ev_charging_points_df.rename(columns={'value': 'charging_points'}, inplace=True)

    exclude_list = ['Europe', 'EU27', 'Rest of the world', 'World']
    ev_charging_points_df = ev_charging_points_df[~ev_charging_points_df['region'].isin(exclude_list)]

    return ev_charging_points_df


In [343]:
powertrain_years_df = pd.read_csv(ev_charging_points_url)
powertrain_years_df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available slow,2019,charging points,1700.0
...,...,...,...,...,...,...,...,...
745,World,Historical,EV charging points,EV,Publicly available fast,2021,charging points,560000.0
746,World,Historical,EV charging points,EV,Publicly available fast,2022,charging points,900000.0
747,World,Historical,EV charging points,EV,Publicly available slow,2022,charging points,1800000.0
748,World,Historical,EV charging points,EV,Publicly available slow,2023,charging points,2500000.0


In [344]:
# giveme only the rows where region is "World"
world_df = powertrain_years_df[powertrain_years_df['region'] == 'World']
world_df

,region,category,parameter,mode,powertrain,year,unit,value
722,World,Historical,EV charging points,EV,Publicly available fast,2010,charging points,310.0
723,World,Historical,EV charging points,EV,Publicly available slow,2010,charging points,3700.0
724,World,Historical,EV charging points,EV,Publicly available slow,2011,charging points,10000.0
725,World,Historical,EV charging points,EV,Publicly available fast,2011,charging points,830.0
726,World,Historical,EV charging points,EV,Publicly available fast,2012,charging points,1500.0
727,World,Historical,EV charging points,EV,Publicly available slow,2012,charging points,28000.0
728,World,Historical,EV charging points,EV,Publicly available slow,2013,charging points,41000.0
729,World,Historical,EV charging points,EV,Publicly available fast,2013,charging points,2600.0
730,World,Historical,EV charging points,EV,Publicly available fast,2014,charging points,13000.0
731,World,Historical,EV charging points,EV,Publicly available slow,2014,charging points,89000.0


In [345]:
# drop the region, category, parameter, unit, mode columns
world_df = world_df.drop(columns=['region', 'category', 'parameter', 'unit', 'mode'])

In [346]:
world_df

,powertrain,year,value
722,Publicly available fast,2010,310.0
723,Publicly available slow,2010,3700.0
724,Publicly available slow,2011,10000.0
725,Publicly available fast,2011,830.0
726,Publicly available fast,2012,1500.0
727,Publicly available slow,2012,28000.0
728,Publicly available slow,2013,41000.0
729,Publicly available fast,2013,2600.0
730,Publicly available fast,2014,13000.0
731,Publicly available slow,2014,89000.0


In [347]:
# change the name of the value column to charging_points
world_df.rename(columns={'value': 'charging_points'}, inplace=True)
world_df
# group by powertrain and year and sum the charging points
world_df.groupby(['powertrain', 'year'])['charging_points'].sum()

powertrain               year
Publicly available fast  2010        310.0
                         2011        830.0
                         2012       1500.0
                         2013       2600.0
                         2014      13000.0
                         2015      25000.0
                         2016      72000.0
                         2017     110000.0
                         2018     140000.0
                         2019     260000.0
                         2020     380000.0
                         2021     560000.0
                         2022     900000.0
                         2023    1400000.0
Publicly available slow  2010       3700.0
                         2011      10000.0
                         2012      28000.0
                         2013      41000.0
                         2014      89000.0
                         2015     150000.0
                         2016     240000.0
                         2017     310000.0
                        

In [348]:
def load_world_data():
    ev_charging_points_url = "https://api.iea.org/evs?parameters=EV-charging-points&category=Historical&mode=EV&csv=true"

    powertrain_years_df = pd.read_csv(ev_charging_points_url)

    powertrain_years_df = powertrain_years_df[powertrain_years_df['region'] == 'World']    

    powertrain_years_df = powertrain_years_df.drop(columns=['region', 'category', 'parameter', 'unit', 'mode'])

    powertrain_years_df.rename(columns={'value': 'charging_points'}, inplace=True) 

    return powertrain_years_df


In [349]:
load_world_data()

,powertrain,year,charging_points
722,Publicly available fast,2010,310.0
723,Publicly available slow,2010,3700.0
724,Publicly available slow,2011,10000.0
725,Publicly available fast,2011,830.0
726,Publicly available fast,2012,1500.0
727,Publicly available slow,2012,28000.0
728,Publicly available slow,2013,41000.0
729,Publicly available fast,2013,2600.0
730,Publicly available fast,2014,13000.0
731,Publicly available slow,2014,89000.0


In [351]:
ev_charging_points_df[ev_charging_points_df["year"] == 2023]

,region,powertrain,year,charging_points
10,Australia,Publicly available fast,2023,660.0
11,Australia,Publicly available slow,2023,2100.0
36,Austria,Publicly available fast,2023,2500.0
37,Austria,Publicly available slow,2023,15000.0
58,Belgium,Publicly available fast,2023,2000.0
...,...,...,...,...
673,Turkiye,Publicly available slow,2023,7000.0
697,United Kingdom,Publicly available fast,2023,10000.0
698,United Kingdom,Publicly available slow,2023,43000.0
720,USA,Publicly available fast,2023,43000.0
